In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def cv_show(name, img):
    cv2.startWindowThread()
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [3]:
img = cv2.imread('模版.png')
cv_show('模版', img)

In [4]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv_show('模版_灰度图', img_gray)

In [5]:
img_binary = cv2.threshold(img_gray, 10, 255, cv2.THRESH_BINARY_INV)[1]
cv_show('模版_二值图', img_binary)

In [6]:
# 轮廓检测
## cv2.findContours()函数应输入二值图（即黑白图），cv2.RETR_EXTERNAL只检测外轮廓，cv2.CHAIN_APPROX_SIMPLE只保留终点坐标
## cv2.findContours()函数返回两个值，一个是轮廓本身，还有一个是每条轮廓对应的属性
refCnts, hierarchy = cv2.findContours(img_binary.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img, refCnts, -1, (0,0,255), 3)
cv_show('img', img)
print(np.array(refCnts).shape)

(10,)


## 模版处理方法：

In [7]:
# 定义轮廓排序函数
def sort_contours(cnts, method='left-to-right'):
    reverse = False
    i = 1
    
    if method == 'right-to-left' or method == 'bottom-to-top':
        reverse = True
        
    if method == 'left-to-right' or method == 'top-to-bottom':
        i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts] #找到最小外接矩形,返回包含x,y,h,w四个值的tuple
    # zip(cnts,boundingBoxes）位置上指可迭代的元素；key是指按什么方式进行排序，这里key=lambda b:b[1][i] b只
    # 是一个变量名，可以是任何一个变量，b[1][i]指按第二元素中的第i个元素就行排序也就是按boundingBoxes中的第i个元素
    # 进行排序；reverse是反转，false为不反转，true为反转。
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                key = lambda b: b[1][i], reverse=reverse))
    return cnts, boundingBoxes

In [8]:
# 轮廓排序
refCnts = sort_contours(refCnts, method='left-to-right')[0]

In [9]:
# 遍历每个轮廓
digits = {}

for (i, c) in enumerate(refCnts):
    # 计算外接矩形，并且resize大小
    (x, y, w, h) = cv2.boundingRect(c)
    roi = img_gray[y:y + h, x:x + w]
    roi = cv2.resize(roi, (57, 88))
    digits[i] = roi
    cv_show('roi', roi)

In [10]:
digits.items()

dict_items([(0, array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 252, ..., 255, 255, 255],
       [255, 248, 170, ..., 208, 255, 255],
       ...,
       [255, 224, 108, ..., 152, 245, 255],
       [255, 255, 229, ..., 247, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)), (1, array([[246, 172,  83, ..., 255, 255, 255],
       [142,  33,   6, ..., 255, 255, 255],
       [ 38,   5,   0, ..., 255, 255, 255],
       ...,
       [ 20,   2,   0, ...,   0,   0,   1],
       [ 88,  13,   1, ...,   0,   4,  21],
       [217, 104,  28, ...,   9,  52, 148]], dtype=uint8)), (2, array([[221, 130,  57, ..., 255, 255, 255],
       [ 73,  13,   3, ..., 255, 255, 255],
       [  7,   1,   0, ..., 218, 255, 255],
       ...,
       [  0,   0,   0, ...,   0,   1,  13],
       [  0,   0,   0, ...,   1,  11,  73],
       [  0,   0,   0, ...,  23,  92, 205]], dtype=uint8)), (3, array([[246, 175,  86, ..., 255, 255, 255],
       [142,  33,   6, ..., 246, 255, 255],
       [

## 输入图像处理：

In [11]:
image = cv2.imread('卡2.png')
image = cv2.resize(image, (300,189))
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv_show('image_gray', image_gray)

In [12]:
# 礼帽操作，突出图像更明亮的区域
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,3))# 初始化两个卷积核
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

tophat = cv2.morphologyEx(image_gray, cv2.MORPH_TOPHAT, rectKernel)
cv_show('tophat', tophat)

In [13]:
grad_x = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)# ksize=-1相当于3*3的卷积核
grad_y = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)
grad = cv2.addWeighted(grad_x, 0.5, grad_y, 0.5, 0)

In [14]:
grad_x = np.absolute(grad_x)
# cv_show('grad_x', grad_x)
(minVal, maxVal) = (np.min(grad_x), np.max(grad_x))
grad_x = (255 * ((grad_x - minVal) / (maxVal - minVal))) # 归一化
grad_x = grad_x.astype('uint8')

In [15]:
print(np.array(grad_x).shape)
cv_show('grad_x', grad_x)

(189, 300)


In [16]:
# 通过闭操作（先膨胀，后腐蚀），将数字连在一起
grad_x = cv2.morphologyEx(grad_x, cv2.MORPH_CLOSE, rectKernel)
cv_show('grad_x', grad_x)
# 二值化处理，THRESH_OTSU会自动寻找合适的阈值，适合双峰，需把阈值参数设置为0
thresh = cv2.threshold(grad_x, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv_show('thresh', thresh)

In [17]:
# 再进行一次闭操作
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
cv_show('thresh', thresh)

In [18]:
# 计算轮廓
threshCnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                                                 cv2.CHAIN_APPROX_SIMPLE)
cnts = threshCnts
cur_img = image.copy()
cv2.drawContours(cur_img, cnts, -1, (0,0,255), 3)
cv_show('img', cur_img)

In [19]:
# 遍历轮廓
locs = []

for (i, c) in enumerate(cnts):
    # 计算矩形
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    
    # 选择合适的区域，根据实际任务来
    if ar > 2.5 and ar < 4.0:
        if(w > 40 and w < 55) and (h > 10 and h < 20):
            locs.append((x, y, w, h))

In [ ]:
# 将轮廓从左到右排序
locs = sorted(locs, key=lambda x:x[0])
output = []

In [ ]:
# 遍历每个轮廓中的数字
for (i, (gx, gy, gw, gh)) in enumerate(locs):
    groupOutput = []
    # 根据坐标提取每个轮廓
    group = image_gray[gy - 5: gy + gh +5, gx - 5: gx + gw +5]
#     cv_show('group', group)
    
    # 预处理
    group = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1] # 二值化
#     cv_show('group', group)
    # 计算每一组的轮廓
    digitCnts, hierarchy = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 排序
    digitCnts = sort_contours(digitCnts, method ='left-to-right')[0]
    
    # 计算每个轮廓中的每个数值
    for c in digitCnts:
        (x, y, w, h) = cv2.boundingRect(c)
        roi = group[y:y + h, x:x + w]
        roi = cv2.resize(roi, (57, 88))
#         cv_show('roi', roi)

        # 计算匹配得分
        scores = []

        # 在模板中计算每一个得分
        for (digit, digitROI) in digits.items():
            # 模板匹配
            result = cv2.matchTemplate(roi, digitROI, cv2.TM_CCOEFF)
            print(cv2.minMaxLoc(result))
            input()
            (_, score, _, _) = cv2.minMaxLoc(result)
            scores.append(score)
            print(scores)

        groupOutput.append(str(np.argmax(scores)))
        print(groupOutput)
        input()

    cv2.rectangle(image, (gx - 5, gy - 5), (gx + gw + 5, gy + gh + 5), (0,0,255, 1))
    cv2.putText(image, ''.join(groupOutput), (gx, gy - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,255), 2)
    output.extend(groupOutput)

In [89]:
# print('Credit Card Type:{}'.format(FIRST_NUMBER[output[0]]))
print('Credit Card #:{}'.format(''.join(output)))
cv_show('Image', image)

Credit Card #:1141454114454767
